# xmon1 <-> xmon2

In [2]:
import numpy as np
import numpy.linalg as alg
import scqubits as scq

In [3]:
hbar = 1.054e-34
h = hbar*2*np.pi
qe = 1.609e-19
flux_quant = h/2/qe

# Unit conversions
MHz = 10.0**(-3)
GHz = 1.0
kHz = 10.0**(-6)
us = 10.0**3
ns = 1.0

In [4]:
# F
Cj1     = 100.282e-15 # pad1 <-> gnd
Cj2     = 82.822e-15 # pad2 <-> gnd
Cq1q2   = 2.81e-15 # pad1 <-> pad2

C_q1_res1 = 10.2e-15 # F # pad1 <-> res1
C_q2_res2 = 9.4e-15 # F # pad2 <-> res2

# GET ZO FROM CPW MATHEMATICA
Zo = 52.7066 # Ohm
res1_freq = 6.1 # GHz
res2_freq = 7.4 # GHz

# H
# Lr1 = 1/(2*np.pi * res1_freq)**2/Cr1
# Lr2 = 1/(2*np.pi * res2_freq)**2/Cr2
# print(Lr * 1e9, 'nH')
Lj1 = 14e-9
Lj2 = 9.0e-9

c_matrix = np.array([
    [Cj1+Cq1q2, -Cq1q2],
    [-Cq1q2,    Cj2+Cq1q2],
])

inv_l_matrix = np.array([ # DON'T FORGET INDUCTANCES SHOULD BE IN DENOMINATOR
    [1/Lj1,   0],
    [0,     1/Lj2],
])

print('Check symmetric:', np.allclose(c_matrix, c_matrix.T), np.allclose(inv_l_matrix, inv_l_matrix.T))

Check symmetric: True True


### simultaneous diagonalization

In [5]:
inv_c_matrix = alg.inv(c_matrix)

# Look for col with max in Q1_NODE -> use q_modes to reference in EC, EJ
# IDK WHY BUT THIS DOESN'T WORK
# evals_c, u1 = alg.eigh(inv_c_matrix)
# u1 = np.array([u1[i]/np.sqrt(evals_c[i]) for i in range(len(evals_c))])
# evals, evecs = alg.eigh(u1.transpose() @ inv_l_matrix @ u1)
# print('Modes (cols are evecs)\n', np.around(evecs, 3))
# q_modes = np.array([np.argmax(np.abs(evecs[i,:])) for i in range(4)])
# print('Qubit modes were mapped', f'q1: mode {q_modes[0]},', f'q2: mode {q_modes[1]},', f'q3: mode {q_modes[2]},', f'q4: mode {q_modes[3]}')
# old_evecs = evecs.copy()
# for i in range(4):
#     evecs[:,i] = old_evecs[:,q_modes[i]]
# # evals = np.array([evals[q_modes[i]] for i in range(4)])
# ECs = qe**2/2/h * evecs.transpose() @ inv_c_matrix @ evecs
# EJs = flux_quant**2/h/4/np.pi**2 * evecs.transpose() @ inv_l_matrix @ evecs

ECs = qe**2/2/h * inv_c_matrix
EJs = flux_quant**2/h/4/np.pi**2 * inv_l_matrix

print(ECs*1e-9)
Zs = np.sqrt(8*np.diag(ECs)/np.diag(EJs))
gs = np.array([[4*ECs[i,j]/np.sqrt(Zs[i]*Zs[j]) for i in range(len(c_matrix))] for j in range(len(c_matrix))])

ECs = 1e-9 * np.diag(ECs)
EJs = 1e-9 * np.diag(EJs)
g_q1_q2 = 1e-9 * gs[0,1]

# AFTER HERE EVERYTHING IS INDEXED SO QUBIT NUMBER = INDEX AND EVERYTHING IS IN GHz

print('EC values (GHz)\n', *ECs)
print('EJ values (GHz)\n', *EJs)
print('impedances (unitless)\n', *Zs)
print('q-q couplings g12, g23, g24 (MHz)\n', g_q1_q2*1e3)


[[0.20211559 0.00734633]
 [0.00734633 0.25309547]]
EC values (GHz)
 0.20211559303880722 0.2530954671075789
EJ values (GHz)
 11.570726476782394 17.99890785277261
impedances (unitless)
 0.3738217704072386 0.3354008047216689
q-q couplings g12, g23, g24 (MHz)
 82.98815570123774


In [6]:
transmon1_consts = dict(
    EC=ECs[0],
    EJ=11.9203,
)

transmon2_consts = dict(
    EC=ECs[1],
    EJ=17.8805,
)

transmon1 = scq.Transmon(
    **transmon1_consts,
    ng=0,
    ncut = 110,
    truncated_dim = 5
)

transmon2 = scq.Transmon(
    **transmon2_consts,
    ng=0,
    ncut = 110,
    truncated_dim = 5
)

evals_count = 5
cutoff = evals_count

evals1, evecs1 = transmon1.eigensys(evals_count=evals_count)
evals1 = evals1 - evals1[0] 
wq1 = evals1[1]

evals2, evecs2 = transmon2.eigensys(evals_count=evals_count)
evals2 = evals2 - evals2[0]
wq2 = evals2[1]

alpha1 = evals1[2]-2*evals1[1]
alpha2 = evals2[2]-2*evals2[1]

print('qubit freq 1:', wq1, 'alpha/2pi:', alpha1)
print('qubit freq 2:', wq2, 'alpha/2pi:', alpha2)

qubit freq 1: 4.177461420308671 alpha/2pi: -0.22904819869831172
qubit freq 2: 5.751820449688871 alpha/2pi: -0.28308347422669655


In [7]:

g_q1_res1 =  C_q1_res1/(C_q1_res1+Cj1) * res1_freq * np.sqrt(2*Zo/(h/qe**2))
g_q2_res2 =  C_q2_res2/(C_q2_res2+Cj2) * res2_freq * np.sqrt(2*Zo/(h/qe**2))

# g_q1_res1 = 50e6
delta1 = wq1-res1_freq
delta2 = wq2-res2_freq
stark_res1 = g_q1_res1**2/(delta1) * alpha1/(delta1+alpha1)
stark_res2 = g_q2_res2**2/(delta2) * alpha2/(delta1+alpha2)
print('g q1<->res1:', 1e3*g_q1_res1, 'MHz\tdelta wr-wq:', 1e3*delta1, 'MHz\tstark:', 1e3*stark_res1, 'MHz')
print('g q2<->res2:', 1e3*g_q2_res2, 'MHz\tdelta wr-wq:', 1e3*delta2, 'MHz\tstark:', 1e3*stark_res2, 'MHz')

g q1<->res1: 38.33148026520377 MHz	delta wr-wq: -1922.538579691329 MHz	stark: -0.08135872465515642 MHz
g q2<->res2: 53.221966298829926 MHz	delta wr-wq: -1648.1795503111289 MHz	stark: -0.22057725676679432 MHz
